# Analyse news from snopes



In [1]:
#Importar as bibliotecas necessarias para a leitura dos posts

import json
import pandas as pd
from matplotlib.font_manager import json_load

import readNews
import sentimentAnalysis

### Read the news

In [2]:
def remove_values_from_list(the_list, val):
    return [value for value in the_list if value != val]

def clean (ListSentences):
    
    #limpeza '
    l = [s.replace("\"","'") for s in ListSentences]
    
    #Separar em frases + pequenas
    sentences = []
    for i in l:
        if "Polígrafo" in i:
            continue
        sentences = sentences + i.split('.')

    return remove_values_from_list(sentences, "\'\'")

### Automatizar o processo


        1) Extrair conteudo do url da noticia
        2) Divir o conteudo extraido por frases simples (split pelo .)
        3) Realizar analise de sentimentos nas frases;
        4) Identificar a frase com maios percentagem de tristeza e surpresa,
        5) Guardar resultados num csv.

In [3]:
 ### Recolher os links que se pretende analisar
    
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re

req = Request("https://www.snopes.com/fact-check/")
html_page = urlopen(req)

soup = BeautifulSoup(html_page, "lxml")

links = []
for link in soup.findAll('a'):
    if type (link.get('href')) == str and (len( link.get('href'))) >  50:  
        links.append(link.get('href'))


In [18]:
list_urls = links[:-1]
list_urls

['https://www.snopes.com/fact-check/octopus-fossil-named-biden/',
 'https://www.snopes.com/fact-check/shoes-foil-spy-hunters/',
 'https://www.snopes.com/fact-check/bill-clinton-meeting-jfk/',
 'https://www.snopes.com/fact-check/russia-sanction-republicans/',
 'https://www.snopes.com/fact-check/disney-child-trafficking/',
 'https://www.snopes.com/fact-check/putin-deepfake-russian-surrender/',
 'https://www.snopes.com/fact-check/zelenskyy-nazi-iron-cross/',
 'https://www.snopes.com/fact-check/ukraine-were-not-gonna-take-it/',
 'https://www.snopes.com/fact-check/pelosi-read-poem-by-bono-ukraine/',
 'https://www.snopes.com/fact-check/reuters-russian-owned/',
 'https://www.snopes.com/fact-check/ukraine-stamp-russian/',
 'https://www.snopes.com/fact-check/marjorie-taylor-greene-zelenskyy/',
 'https://www.snopes.com/fact-check/mayim-bialik-cbd-allegations/',
 'https://www.snopes.com/fact-check/apple-juice-penis-size/',
 'https://www.snopes.com/news/2022/03/16/zelenskyy-deepfake-shared/',
 'ht

In [21]:
language = 'en'
def extractor (url):
  #  summary = readNews.main(url, "summary").encode('utf8').decode().split('\n')
  #  title = readNews.main(url, "title").encode('utf8').decode().split('\n')
 #   newsSentences = title + summary
    text = readNews.main(url, "text").encode('utf8').decode().split('\n')
    return text 

def sentiment_analysis(text, language):  
    
    ## Limpar o texto, transformar paragrafos em frases
    text =clean(text)
    
    status_code = 0 ## Valor inicial 
    
    while (status_code != 200):
        emotions, status_code = sentimentAnalysis.main(text,language)
    playload_emotions = json.loads(emotions)
    
    return playload_emotions
    

    
def main ():
    for url in list_urls:
        text =  extractor (url)
    
        playload_emotions = sentiment_analysis(text, language)

        ## Ordenar as frases por aquela que tem maior percentagem de sadness e surprise
        sortedByEmotion = {k: v for k, v in sorted(playload_emotions.items(), key=lambda item: (item[1]['emotions']['sadness'] + item[1]['emotions']['surprise']), reverse=True)}
     
        data = pd.read_csv('snopes_news.csv')
        
        row = {'url': url, "language": language, "text": list (sortedByEmotion) [0], "surprise": list(sortedByEmotion.items())[0][1]['emotions']['surprise'] , 'sadness':list(sortedByEmotion.items())[0][1]['emotions']['sadness']}
        print(row)
        # Adicionar a nova linha ao dataset
        data = data.append(row,  ignore_index=True)

        #Retirar colunaas que estão a mais
        data = data[['url','language','text', "surprise" ,'sadness' ]]
        
        data.to_csv('snopes_news.csv')

In [22]:
main()

{'url': 'https://www.snopes.com/fact-check/octopus-fossil-named-biden/', 'language': 'en', 'text': 'Scientists named an extinct relative of the octopus after U', 'surprise': 0.0, 'sadness': 0.2}
{'url': 'https://www.snopes.com/fact-check/shoes-foil-spy-hunters/', 'language': 'en', 'text': 'The story goes about a Herr Wolf from a shoe import firm who got tired about people always complaining about new shoe collections so he made these shoes with the heels at the front', 'surprise': 0.0, 'sadness': 0.5}
{'url': 'https://www.snopes.com/fact-check/bill-clinton-meeting-jfk/', 'language': 'en', 'text': ' Parade quoted a longer excerpt, giving more detail about where Clinton was when he got the news:', 'surprise': 0.33, 'sadness': 0.0}
{'url': 'https://www.snopes.com/fact-check/russia-sanction-republicans/', 'language': 'en', 'text': ' In the March 15 news release, the Foreign Ministry warned:', 'surprise': 0.25, 'sadness': 0.0}
{'url': 'https://www.snopes.com/fact-check/disney-child-traffick